# Building an AI Agent from Scratch

graph TD
    A[User Question] --> B[Initialize TravelAgent]
    B --> C{Extract Budget?}
    C -- Yes --> D[Set Budget]
    C -- No --> E[Continue]
    D --> E
    
    E --> F[Generate AI Response]
    F --> G{Does Response Contain an Action?}
    G -- Yes --> H[Parse Action]
    H --> I{Action Type?}
    
    I -- convert_currency --> J[Call Currency API]
    J --> K[Receive Conversion Data]
    K --> M[Store in Memory & Continue]

    I -- get_weather --> L[Call Weather API]
    L --> N[Receive Weather Data]
    N --> M

    I -- translate_text --> O[Call Translation Model]
    O --> P[Receive Translation]
    P --> M
    
    G -- No --> Q[Provide Final Answer]
    M --> R[Generate Next Thought & Action]
    R --> G
    Q --> S[End]


### Install required libraries

In [5]:
!pip install -q openai sentencepiece transformers

### Load the environment variables

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
os.environ.get('OPENAI_API_KEY')
print("API Key Loaded", os.environ.get('OPENAI_API_KEY') is not None)

API Key Loaded True


### Import Necessary Modules

In [2]:
import openai
import re
import os
import requests
from datetime import datetime, timedelta

### Testing our Configuration

In [8]:
from openai import OpenAI
client = OpenAI()
MODEL='gpt-4o-mini'
prompt = 'Write something short but funny.'
chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            'role':'user', 'content':prompt
        }
    ]
)
chat_completion.choices[0].message.content

'Why did the scarecrow win an award? Because he was outstanding in his field!'

### Creating the Agent Class

In [9]:
class TravelAgent:
    def __init__(self, system_prompt, initial_budget=1000):
        self.system_prompt = system_prompt
        self.budget = initial_budget
        self.messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'system', 'content': f'The current budget is ${self.budget}.'}
        ]
        self.memory = []

    def __call__(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        response = self.execute()
        self.messages.append({"role": "assistant", "content": response})
        return response

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=self.messages,
            temperature=0.7
        )
        return completion.choices[0].message.content

    def add_to_memory(self, action, result):
        self.memory.append(f"Action: {action}, Result: {result}")

    def set_budget(self, amount):
        self.budget = amount
        self.messages.append({'role': 'system', 'content': f'The current budget is ${self.budget}.'})

    def get_budget(self):
        return self.budget

# Tools for the agent

### Currency Converter Tool

In [4]:
def convert_currency(amount, from_currency, to_currency):
    url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response = requests.get(url)
    data = response.json()
    rate = data['rates'][to_currency]
    converted_amount = amount * rate
    return round(converted_amount, 2)

In [8]:
convert_currency(150, "USD", "NGN")

226093.5

### Weather Tool

In [12]:
def get_weather(location: str, days_ahead: int = 0) -> str:
    """
    This function uses the OpenWeatherMap API to get the current weather for a given location.
    """ 

    api_key = os.environ.get("OPENWEATHERMAP_API_KEY")
    if not api_key:
        raise Exception("OPENWEATHERMAP_API_KEY environment variable not set.")

    if days_ahead > 0:
        target_date = (datetime.now() + timedelta(days=days_ahead)).strftime('%Y-%m-%d')
    else:
        target_date = None

    base_url = "http://api.openweathermap.org/data/2.5/weather" 

    params = {
        "q": location,
        "appid": api_key,
        "units": "metric"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        if target_date:
            return f"The weather in {data['name']}, {data['sys']['country']} on {target_date} is currently {data['weather'][0]['description']} with a temperature of {data['main']['temp']}°C. The humidity is {data['main']['humidity']}% and the wind speed is {data['wind']['speed']} m/s."
        else:
            return f"The weather in {data['name']}, {data['sys']['country']} is currently {data['weather'][0]['description']} with a temperature of {data['main']['temp']}°C. The humidity is {data['main']['humidity']}% and the wind speed is {data['wind']['speed']} m/s."
    else:
        raise Exception(f"Error fetching weather data: {response.status_code}")

In [16]:
get_weather('Lefke',0)

'The weather in Lefka, CY is currently scattered clouds with a temperature of 8.22°C. The humidity is 75% and the wind speed is 1.75 m/s.'

### Translation Tool

In [17]:
from transformers import pipeline

def translate_text(text, target_language):
    try:
        translator = pipeline("translation", model="facebook/m2m100_418M")
        translation = translator(text, src_lang="en", tgt_lang=target_language)[0]['translation_text']
        return translation
    except Exception as e:
        print(f"An error occurred during translation: {e}")
        return "Translation failed."

C:\Users\valen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [18]:
pip install  PyTorch TensorFlow 


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.3/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.8/390.3 MB 2.8 MB/s eta 0:02:19
   ---------------------------------------- 0.8/390.3 MB 2.8 MB/s eta 0:02:19
   ---------------------------------------- 1.0/390.3 MB 1.2 MB/s eta 0:05:33
   ---------------------------------------- 1.0/390.3 MB 1.2 MB/s eta 0:05:33
   ----

  error: subprocess-exited-with-error
  
  × Building wheel for PyTorch (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "C:\Users\valen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "C:\Users\valen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\valen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pip\_vendor\pyproject_hooks\_in_process\_i


   ---------------------- --------------- 229.9/390.3 MB 612.0 kB/s eta 0:04:23
   ---------------------- --------------- 229.9/390.3 MB 612.0 kB/s eta 0:04:23
   ---------------------- --------------- 230.2/390.3 MB 598.9 kB/s eta 0:04:28
   ---------------------- --------------- 230.2/390.3 MB 598.9 kB/s eta 0:04:28
   ---------------------- --------------- 230.2/390.3 MB 598.9 kB/s eta 0:04:28
   ---------------------- --------------- 230.4/390.3 MB 592.9 kB/s eta 0:04:30
   ---------------------- --------------- 230.4/390.3 MB 592.9 kB/s eta 0:04:30
   ---------------------- --------------- 230.7/390.3 MB 591.3 kB/s eta 0:04:30
   ---------------------- --------------- 230.7/390.3 MB 591.3 kB/s eta 0:04:30
   ---------------------- --------------- 230.7/390.3 MB 591.3 kB/s eta 0:04:30
   ---------------------- --------------- 230.9/390.3 MB 583.4 kB/s eta 0:04:34
   ---------------------- --------------- 230.9/390.3 MB 583.4 kB/s eta 0:04:34
   ---------------------- -------------

In [11]:
english_text = "Hello, where is the nearest resturant?"
target_language = "fr"

translated_text = translate_text(english_text, target_language)
print(f"Original: {english_text}")
print(f"Translated: {translated_text}")

pytorch_model.bin:   4%|3         | 73.4M/1.94G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/facebook/m2m100_418M/d907ea45e4e4b9db163382a6674f6218b3c59566fe06d77f4055c208b4e87ed1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1738866392&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODg2NjM5Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9mYWNlYm9vay9tMm0xMDBfNDE4TS9kOTA3ZWE0NWU0ZTRiOWRiMTYzMzgyYTY2NzRmNjIxOGIzYzU5NTY2ZmUwNmQ3N2Y0MDU1YzIwOGI0ZTg3ZWQxP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=DF9J8cKTYR8EvrIIAGvfgUoJGNAHCAkhLI8w7weaW1bjj5xX%7EBg0lTVQnKDPOIW0GPW-Wrz%7Eu%7EpkR3AQBG6klf75JXBWWClqdGGGbEhSgjPkxucZhKUKHPNP3QI8rolGHIrQDalKz35lcQ95ADaXActkZeLdV6BSLyuZtd-z4wFt3dq4D5O%7EO6Hvq94SdqPzvV7y6Tkb0ShWWsH4U7fwgF91HLZno3JHKxZuR4xdQ1CxmbUXVU7HOG2zQgRPAdTQhBfIYOzqLH1yI0aLTWxlLUxSNfcU3hVlUwRr84XJewF6Ctc-I5umw%7E

pytorch_model.bin:  19%|#8        | 367M/1.94G [00:00<?, ?B/s]

KeyboardInterrupt: 

# Prompt

In [15]:
travel_agent_prompt = """
You are a Travel Planning Assistant. Your role is to help users plan their trips by providing information about destinations, 
currency conversion, weather forecasts, and language translation. 

You run in a loop of Thought, Action, PAUSE, Observation.
Use Thought to describe your reasoning about the user's request.
Use Action to perform one of the available actions, then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

1. convert_currency:
   e.g., convert_currency: 100, USD, EUR
   Converts the given amount from one currency to another.

2. get_weather:
   e.g., get_weather: Paris, 3
   Gets the weather forecast for the specified city and number of days ahead (0 for today).

3. translate_text:
   e.g., translate_text: Hello, how are you?, fr
   Translates the given text to the specified target language (use language codes like 'fr' for French).

Always consider the user's budget when making suggestions. The current budget will be provided in the system messages.

Example session:

User: I'm planning a trip to Paris next week. What should I know?

Thought: I should check the weather in Paris for next week and provide some basic French phrases.
Action: get_weather: Paris, 7
PAUSE

Observation: Partly cloudy, 22°C

Thought: Now that I have the weather information, I should provide a simple French phrase and its translation.
Action: translate_text: Hello, how are you?, fr
PAUSE

Observation: Bonjour, comment allez-vous ?

Answer: Great! For your trip to Paris next week, here's what you should know:

1. Weather: It's expected to be partly cloudy with a temperature of 22°C (72°F). 
   Pack accordingly with light layers and maybe a light jacket for evenings.

2. Language: While many people in Paris speak some English, it's always appreciated if you try some basic French. 
   Here's a useful phrase: "Hello, how are you?" in French is "Bonjour, comment allez-vous?"

Remember to consider the budget for your trip and let me know if you need any currency conversion or have any other questions
about your Paris adventure!
"""

# Building the AI Travel Agent

### Dictionary of available actions

In [16]:
known_actions = {
    'convert_currency': convert_currency,
    'get_weather': get_weather,
    'translate_text': translate_text
}

### AI Travel Agent

In [17]:
def plan_trip(question, max_turns=5):
    """
    Plans a trip based on the given question and executes a series of actions to gather necessary information.
    Args:
        question (str): The initial question or prompt that describes the trip requirements.
        max_turns (int, optional): The maximum number of turns or iterations to attempt in planning the trip. Defaults to 5.
    Returns:
        None
    The function performs the following steps:
    1. Initializes a TravelAgent with a predefined prompt.
    2. Extracts the budget from the question if specified and sets it in the agent.
    3. Iteratively processes the agent's responses up to a maximum number of turns.
    4. Parses actions from the agent's responses and executes known actions.
    5. Handles specific actions such as currency conversion, weather retrieval, and text translation.
    6. Updates the agent's memory with observations from executed actions.
    7. Handles errors during action execution and provides feedback for retrying.
    Note:
        - The function assumes the existence of a `TravelAgent` class and a `known_actions` dictionary mapping action names to their corresponding functions.
        - The function prints the results and observations to the console.
    """
    # Initialize the TravelAgent with the predefined prompt
    agent = TravelAgent(travel_agent_prompt)
    
    # Extract the budget from the question if specified and set it in the agent
    budget_match = re.search(r'\$(\d+)', question)
    if budget_match:
        budget = int(budget_match.group(1))
        agent.set_budget(budget)
    
    # Set the initial prompt to the question
    next_prompt = question
    
    # Compile a regular expression to match actions
    action_re = re.compile(r'^Action: (\w+): (.+)$')
    
    # Iterate up to the maximum number of turns
    for i in range(max_turns):
        # Get the agent's response to the current prompt
        result = agent(next_prompt)
        print(result)
        
        # Parse actions from the agent's response
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        
        if actions:
            # Extract the action and its input
            action, action_input = actions[0].groups()
            
            # Check if the action is known
            if action not in known_actions:
                print(f'Unknown action: {action}: {action_input}')
                continue
            
            print(f' -- running {action}: {action_input}')
            
            try:
                # Execute the action and get the observation
                if action == 'convert_currency':
                    amount, from_currency, to_currency = action_input.split(',')
                    observation = known_actions[action](float(amount), from_currency.strip(), to_currency.strip())
                elif action == 'get_weather':
                    location, *days = action_input.split(',')
                    days_ahead = int(days[0]) if days else 0
                    observation = known_actions[action](location.strip(), days_ahead)
                elif action == 'translate_text':
                    if ',' in action_input:
                        text, target_lang = action_input.rsplit(',', 1)
                    else:
                        text, target_lang = action_input.rsplit(None, 1)
                    observation = known_actions[action](text.strip(), target_lang.strip())
                
                # Print the observation and update the agent's memory
                print(f'Observation: {observation}')
                agent.add_to_memory(action, observation)
                next_prompt = f'Observation: {observation}'
            except Exception as e:
                # Handle errors during action execution
                print(f"Error executing action: {e}")
                next_prompt = f"Error: Unable to execute action {action}. Please try again."
        else:
            return

In [18]:
question = "I'm planning a 5-day trip to Dubai next month. My budget is $2000. What should I prepare?"
plan_trip(question)

Thought: I should provide information about the weather in Dubai for next month, as well as some useful Arabic phrases for the trip. Additionally, I should consider the user's budget to suggest activities or accommodations that fit within it. 

Action: get_weather: Dubai, 30 
PAUSE
 -- running get_weather: Dubai, 30 
Observation: The weather in Dubai, AE on 2025-03-08 is currently scattered clouds with a temperature of 17.96°C. The humidity is 72% and the wind speed is 2.57 m/s.
Thought: I have the weather information for Dubai, but it's only for one day. I should provide a general idea of the weather for the month and then translate a few useful phrases to Arabic.

Action: get_weather: Dubai, 5 
PAUSE
 -- running get_weather: Dubai, 5 
Observation: The weather in Dubai, AE on 2025-02-11 is currently scattered clouds with a temperature of 17.96°C. The humidity is 72% and the wind speed is 2.57 m/s.
Thought: It seems that I received the same weather information again. Since the weather 

pytorch_model.bin:   2%|2         | 41.9M/1.94G [00:00<?, ?B/s]

KeyboardInterrupt: 